In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="openai/gpt-oss-120b", temperature=0)

In [3]:
from dataclasses import dataclass

@dataclass
class ColourContext:
    favourite_colour: str = "blue"
    least_favourite_colour: str = "yellow"

In [4]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    context_schema=ColourContext  
)

In [5]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext()
)

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='57a65bfa-5953-413f-b15d-5170662df651'),
              AIMessage(content='I don’t have enough information to know your favorite colour.\u202fWould you like to tell me what it is, or perhaps give some clues so I can guess?', additional_kwargs={'reasoning_content': 'The user asks "What is my favourite colour?" We have no prior context. We cannot infer. We must respond that we don\'t know. According to policy, we can ask for clarification. So respond politely asking.'}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 77, 'total_tokens': 163, 'completion_time': 0.177920811, 'completion_tokens_details': {'reasoning_tokens': 44}, 'prompt_time': 0.00289269, 'prompt_tokens_details': None, 'queue_time': 0.004881509, 'total_time': 0.180813501}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_49bfac06f1', 'service_tier': 'on_demand', 'finish

In [7]:
print(response["messages"][-1].content)

I don’t have enough information to know your favorite colour. Would you like to tell me what it is, or perhaps give some clues so I can guess?


## Accessing Context

In [8]:
from typing import Annotated
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig

@tool
def get_favourite_colour(config: RunnableConfig) -> str:
    """Get the favourite colour of the user"""
    # Accessing the context passed via the 'configurable' field
    context = config.get("configurable", {}).get("context")
    if context:
        return context.favourite_colour
    return "I don't know the favourite colour."

@tool
def get_least_favourite_colour(config: RunnableConfig) -> str:
    """Get the least favourite colour of the user"""
    context = config.get("configurable", {}).get("context")
    if context:
        return context.least_favourite_colour
    return "I don't know the least favourite colour."

In [9]:
agent = create_agent(
    model=model,
    tools=[get_favourite_colour, get_least_favourite_colour],
    context_schema=ColourContext
)

In [10]:
config = {
    "configurable": {
        "thread_id": "session_123",
        "context": ColourContext(
            favourite_colour="blue", 
            least_favourite_colour="yellow"
        )
    }
}

In [11]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    config=config
)

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='05f1da7e-d3b2-441f-ba25-488ec0d9bcba'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to answer using the function get_favourite_colour. The user asks "What is my favourite colour?" We should call the function.', 'tool_calls': [{'id': 'fc_b769b4a0-3fa5-407e-a818-d6476b6b4cf6', 'function': {'arguments': '{}', 'name': 'get_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 149, 'total_tokens': 199, 'completion_time': 0.103196941, 'completion_tokens_details': {'reasoning_tokens': 29}, 'prompt_time': 0.005999419, 'prompt_tokens_details': None, 'queue_time': 0.004750651, 'total_time': 0.10919636}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8634b911ff', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq

In [12]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my least favourite colour?")]},
    config=config
)

pprint(response)

{'messages': [HumanMessage(content='What is my least favourite colour?', additional_kwargs={}, response_metadata={}, id='1b72f326-bc86-45cf-b83a-6bc9ff0e3197'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to answer user\'s question: "What is my least favourite colour?" We have functions to get favourite and least favourite colour. We should call get_least_favourite_colour.', 'tool_calls': [{'id': 'fc_d71c87cc-6c45-4eca-b9da-a8cd5e2a39ca', 'function': {'arguments': '{}', 'name': 'get_least_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 150, 'total_tokens': 210, 'completion_time': 0.285401824, 'completion_tokens_details': {'reasoning_tokens': 37}, 'prompt_time': 0.006640392, 'prompt_tokens_details': None, 'queue_time': 0.004613313, 'total_time': 0.292042216}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_2804ae6b41', 'service_tier': 'on_demand', 'finish_reaso

In [13]:
print(response["messages"][-1].content)

Your least favourite colour is **yellow**.
